# [실습] CNN활용_데이터증강_전이학습_개고양이 분류(Kaggel)_ResNet50_Data_Augmentation

* 데이터출처: https://www.kaggle.com/tongpython/cat-and-dog

In [1]:
# 라이브러리 불러오기
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 모델 학습

### ImageDataGenerator

In [2]:
# 케라스로 ImageDataGenerator 클래스의 객체를 생성하고,
# rescale옵션을 지정해 이미지 각 픽셀의 값을 0~1로 정규화한다.
# 생성된 제너레이터 객체를 image_gen변수에 할당한다.
image_gen = ImageDataGenerator(rescale=(1/255.0))
image_gen

### flow_from_directory함수

In [3]:
# flow_from_directory함수로 폴더에서 이미지를 가져와 제너레이터 객체로 정리
train_dir = '../data/cat_and_dog/training_set/training_set'
valid_dir = '../data/cat_and_dog/test_set/test_set'

train_gen = image_gen.flow_from_directory(train_dir, 
                                          batch_size=32,
                                          target_size=(224, 224),
                                          classes=['cats', 'dogs'],
                                          class_mode='binary',
                                          seed=2023)
valid_gen = image_gen.flow_from_directory(valid_dir, 
                                          batch_size=32,
                                          target_size=(224, 224),
                                          classes=['cats', 'dogs'],
                                          class_mode='binary',
                                          seed=2023)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [4]:
class_labels = ['cat', 'dog']

In [5]:
# 샘플 이미지 출력(첫번째 배치)
batch = next(train_gen)
type(batch)
len(batch)
images, labels = batch[0], batch[1]


In [6]:
images.shape

(32, 224, 224, 3)

In [7]:
print(labels)

[1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0.
 1. 0. 1. 0. 0. 1. 0. 1.]


In [30]:
# 이미지 시각화

plt.figure(figsize=(16,8))
for i in range(0,32):
    ax = plt.subplot(4, 8, i+1)
    plt.axis('off')
    plt.title(labels[i])
    plt.imshow(images[i])
plt.show()

### 모델 훈련

In [8]:
# Sequential API를 활용하여 샘플 모델 생성
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dense, Flatten, Dropout

model = Sequential([
    # Feature Learning
    Conv2D(32, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2,2)),
    BatchNormalization(),

    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2,2)),
    BatchNormalization(),

    Conv2D(128, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2,2)),
    BatchNormalization(),

    # Classifier Learning
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')     # binary로 하였기 때문에 노드가 1개여야 한다.

    
])

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-02-27 11:25:35.516990: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-27 11:25:35.517762: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics='accuracy')

In [19]:
# 모델 훈련
hist = model.fit(train_gen, validation_data=valid_gen, epochs=20, verbose=2)

Epoch 1/20


2023-02-27 11:30:32.166626: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-27 11:30:32.356936: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-27 11:31:30.423339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


251/251 - 64s - loss: 0.2020 - accuracy: 0.9047 - val_loss: 0.7453 - val_accuracy: 0.7963 - 64s/epoch - 253ms/step
Epoch 2/20
251/251 - 62s - loss: 0.2188 - accuracy: 0.9003 - val_loss: 0.7263 - val_accuracy: 0.7800 - 62s/epoch - 248ms/step
Epoch 3/20
251/251 - 63s - loss: 0.1873 - accuracy: 0.9156 - val_loss: 1.0186 - val_accuracy: 0.7558 - 63s/epoch - 249ms/step
Epoch 4/20
251/251 - 63s - loss: 0.1842 - accuracy: 0.9176 - val_loss: 0.6694 - val_accuracy: 0.6911 - 63s/epoch - 249ms/step
Epoch 5/20
251/251 - 62s - loss: 0.1923 - accuracy: 0.9107 - val_loss: 0.9320 - val_accuracy: 0.7815 - 62s/epoch - 246ms/step
Epoch 6/20
251/251 - 62s - loss: 0.1617 - accuracy: 0.9243 - val_loss: 0.7508 - val_accuracy: 0.7954 - 62s/epoch - 247ms/step
Epoch 7/20
251/251 - 62s - loss: 0.1572 - accuracy: 0.9277 - val_loss: 0.8010 - val_accuracy: 0.8047 - 62s/epoch - 246ms/step
Epoch 8/20
251/251 - 62s - loss: 0.1649 - accuracy: 0.9278 - val_loss: 0.9627 - val_accuracy: 0.7800 - 62s/epoch - 245ms/step
Epo

In [ ]:
model.save['255.1_cat_and_dog_without_aug.h5']

In [17]:
model = tf.keras.models.load_model('../data/cat_and_dog/225.1_cat_and_dog_without_aug.h5')

In [22]:
df = pd.DataFrame(hist.history)
df.to_csv('../data/cat_and_dog/255.1_cat_and_dog_without_aug.csv', index=False)
df = pd.read_csv("../data/cat_and_dog/225.1_cat_and_dog_without_aug.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: '../data/cat_and_dog/225.1_cat_and_dog_without_aug.csv'

In [ ]:
# 손실함수, 정확도 시각화
fig, ax = plt.subplots(figsize=(8, 4), nrows=1, ncols=2)

ax[0].plot(range(1, 21), df['loss'], label='loss')
ax[0].plot(range(1,21), df['val_loss'], label='val_loss')
ax[0].legend()
ax[1].plot(range(1, 21), df['accuracy'], label='accuracy')
ax[1].plot(range(1,21), df['val_accuracy'], label='val_accuracy')
ax[1].legend();

###  데이터 증강

In [ ]:
# 이미지 데이터 제너레이터 정의(  Augmentation 적용)
image_gen_aug = ImageDataGenerator(rescale = 1/255,
                                  horizontal_flip = True,
                                  rotation_range=30, 
                                  shear_range=0.15,
                                  zoom_range=0.3)

# flow_from_directory 함수로 폴더에서 이미지 가져와서 제너레이터 객체로 정리
train_gen_aug = image_gen_aug.flow_from_directory(train_dir,
                                                 batch_size=32,
                                                 target_size=(224, 224),
                                                 classes=['cats', 'dogs'],
                                                 class_mode ='binary',
                                                 seed=2020)
valid_gen_aug = image_gen_aug.flow_from_directory(valid_dir,
                                                 batch_size=32,
                                                 target_size=(224, 224),
                                                 classes=['cats', 'dogs'],
                                                 class_mode ='binary',
                                                 seed=2020)

In [ ]:
# 모델 생성


In [ ]:
# 모델 컴파일
model_aug.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), 
                  loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
                 metrics=['accuracy'])

In [ ]:
# 모델 훈련
history_aug = model_aug.fit(train_gen_aug, validation_data = valid_gen_aug, epochs=40)

In [ ]:
model_aug.save('255.1_cnn_catsndogs_data_aug.h5')

In [ ]:
import pandas as pd
df = pd.DataFrame(history_aug.history)
df.to_csv("./output/255.1_데이터증강_history.csv", index=False)
df = pd.read_csv("./output/255.1_데이터증강_history.csv")
df

In [ ]:
# 손실함수, 정확도 그래프 그리기

import matplotlib.pyplot as plt

plt.figure(figsize=(5,5))
plt.plot(range(1, 21), df['loss'], label='train_loss')
plt.plot(range(1, 21), df['val_loss'], label='val_loss');